# Snuffelfiets kwartaalrapportages

In [ ]:
# Generieke imports, variabelen en functies.

from pathlib import Path

import numpy as np
import pandas as pd

from snuffelfiets import inlezen, opschonen, analyse, plotting

api_key = ''  # voeg hier de API key toe
data_directory = Path('~','kwartaalrapportage').expanduser()
year = '2023'
quarter = 'Q1'

quarters = {
    'Q1': [1, 2, 3], 
    'Q2': [4, 5, 6], 
    'Q3': [7, 8, 9], 
    'Q4': [10, 11, 12], 
}
yq = f'{year}_{quarter}'

output_directory = Path(data_directory, yq)
output_directory.mkdir(parents=True, exist_ok=True)

print(f'Analysing quarter {yq}; writing output to {output_directory}.')


In [ ]:
# Read data from the database.
# NOTE: reading in monthly chunks because of serverside errors when trying to pull the full quarter

dfs = []
for i in quarters[quarter]:
    start_datum = f'{year}-{i:02d}-01'
    stop_datum = f'{year+1}-01-01' if i==12 else f'{year}-{i+1:02d}-01'
    df = inlezen.call_api(api_key, start_datum, stop_datum)
    dfs.append(df)
df = pd.concat(dfs)

filename = f'api_gegevens_{yq}.pickle'
p = Path(output_directory, filename)
df.to_pickle(p)

print(f'Read {df.shape[0]} measurements; saved raw data to {filename}.')


In [ ]:
# Drop the errors.
df = opschonen.verwijder_errors(df)


In [ ]:
# Correct units of selected columns.
df = opschonen.correct_units(df)


In [ ]:
# Convert timestamps to datetime objects and add dt columns.
df = analyse.bewerk_timestamp(df, split=True)


In [ ]:
# Split measuremnts into rides and add cycle stat columns.
df = analyse.split_in_ritten(df, t_seconden=1800)


In [ ]:
# Filter the rides.
df = analyse.filter_ritten(df)


In [ ]:
# Print the summary statistics.

def printfun(period, sumstats):

    print(f'\n==== {period} ====\n')

    print(f"Aantal fietsers: {sumstats['fietsers']['N']}\n")

    print(f"{' ':20} {'totaal':12} {'gemiddeld':12} {'topper':12}")
    print(f'-' * 56)
    stat = 'uren'
    print(f"FIETSTIJD [uur]:  {sumstats[stat]['N']:12f} {sumstats[stat]['G']:12f} {sumstats[stat]['M']:12f}")
    stat = 'afstand'
    print(f"AFSTAND    [km]:  {sumstats[stat]['N']:12f} {sumstats[stat]['G']:12f} {sumstats[stat]['M']:12f}")
    stat = 'ritten'
    print(f"RITTEN      [#]:  {sumstats[stat]['N']:12f} {sumstats[stat]['G']:12f} {sumstats[stat]['M']:12f}")

sumstats = analyse.summary_stats(df)
printfun(quarter, sumstats)


In [ ]:
for i, dfm in df.groupby('month'):
    sumstats = analyse.summary_stats(dfm)
    printfun(f'2023{i:02d}', sumstats)
